<a href="https://www.kaggle.com/code/averma111/house-price-pytorch-v1?scriptVersionId=127072136" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split

In [2]:
ROOT_PATH='/kaggle/input/house-prices-advanced-regression-techniques'
train = pd.read_csv(ROOT_PATH+'/train.csv')
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
train.describe(include='all')

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
count,1460.000000,1460.000000,1460,1201.000000,1460.000000,1460,91,1460,1460,1460,...,1460.000000,7,281,54,1460.000000,1460.000000,1460.000000,1460,1460,1460.000000
unique,NaN,NaN,5,NaN,NaN,2,2,4,4,2,...,NaN,3,4,4,NaN,NaN,NaN,9,6,NaN
top,NaN,NaN,RL,NaN,NaN,Pave,Grvl,Reg,Lvl,AllPub,...,NaN,Gd,MnPrv,Shed,NaN,NaN,NaN,WD,Normal,NaN
freq,NaN,NaN,1151,NaN,NaN,1454,50,925,1311,1459,...,NaN,3,157,49,NaN,NaN,NaN,1267,1198,NaN
mean,730.500000,56.897260,NaN,70.049958,10516.828082,NaN,NaN,NaN,NaN,NaN,...,2.758904,NaN,NaN,NaN,43.489041,6.321918,2007.815753,NaN,NaN,180921.195890
std,421.610009,42.300571,NaN,24.284752,9981.264932,NaN,NaN,NaN,NaN,NaN,...,40.177307,NaN,NaN,NaN,496.123024,2.703626,1.328095,NaN,NaN,79442.502883
min,1.000000,20.000000,NaN,21.000000,1300.000000,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,0.000000,1.000000,2006.000000,NaN,NaN,34900.000000
25%,365.750000,20.000000,NaN,59.000000,7553.500000,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,0.000000,5.000000,2007.000000,NaN,NaN,129975.000000
50%,730.500000,50.000000,NaN,69.000000,9478.500000,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,0.000000,6.000000,2008.000000,NaN,NaN,163000.000000
75%,1095.250000,70.000000,NaN,80.000000,11601.500000,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,0.000000,8.000000,2009.000000,NaN,NaN,214000.000000


In [4]:
train.drop(['Id'], axis = 1, inplace = True)
train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
num_rows = len(train.index)
print('The total number of rows in the dataframe is:',num_rows)

The total number of rows in the dataframe is: 1460


In [6]:
print(train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   Alley          91 non-null     object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

## Filling the missing NAN values with mean and mode`

In [7]:
categorical_features = train.select_dtypes(include = ["object","bool"]).columns
numerical_features = train.select_dtypes(include = ["int64","float64"]).columns
numerical_features = numerical_features.drop("SalePrice")

print("Numerical features : " + str(len(numerical_features)))
print("Categorical features : " + str(len(categorical_features)))
train_num = train[numerical_features]
train_cat = train[categorical_features]

Numerical features : 36
Categorical features : 43


In [8]:
# Handle missing values for numerical features by using median as replacement
print("NAs for numerical features in train : " + str(train[numerical_features].isnull().values.sum()))
train[numerical_features] = train[numerical_features].fillna(train[numerical_features].median())
print("Remaining NAs for numerical features in train : " + str(train[numerical_features].isnull().values.sum()))

NAs for numerical features in train : 348
Remaining NAs for numerical features in train : 0


In [9]:
# Use OrdinalEncoder as one-hot encoding results in different columns counts between Model Training and Test 
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
train[categorical_features] = encoder.fit_transform(train[categorical_features])

In [10]:
# Handle missing values for categorical features
print("NANs for categorical features in train : " + str(train[categorical_features].isnull().values.sum()))
train[categorical_features] = train[categorical_features].fillna(method="ffill")
train[categorical_features] = train[categorical_features].fillna(0)
print("Remaining NAs for categorical features in train : " + str(train[categorical_features].isnull().values.sum()))

NANs for categorical features in train : 6617
Remaining NAs for categorical features in train : 0


In [11]:
output_col = [train.columns[-1]]
output_col

['SalePrice']

In [12]:
input_col = train.columns[0:79]
input_col

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'Wo

In [13]:
def datafrme_to_arrays(dataset):
    dataframe = dataset.copy(deep=True)
    
    inputs_array = dataframe[input_col].to_numpy()
    targets_array = dataframe[output_col].to_numpy()
    return inputs_array,targets_array.reshape((1460,1))

In [14]:
inputs_array,targets_array=datafrme_to_arrays(train)
inputs_array,targets_array

(array([[  60.,    3.,   65., ..., 2008.,    8.,    4.],
        [  20.,    3.,   80., ..., 2007.,    8.,    4.],
        [  60.,    3.,   68., ..., 2008.,    8.,    4.],
        ...,
        [  70.,    3.,   66., ..., 2010.,    8.,    4.],
        [  20.,    3.,   68., ..., 2010.,    8.,    4.],
        [  20.,    3.,   75., ..., 2008.,    8.,    4.]]),
 array([[208500],
        [181500],
        [223500],
        ...,
        [266500],
        [142125],
        [147500]]))

In [15]:
import torch
import torchvision

In [16]:
inputs = torch.Tensor(inputs_array)
targets = torch.Tensor(targets_array)

print(inputs)
print(targets)
inputs.shape, targets.shape

tensor([[  60.,    3.,   65.,  ..., 2008.,    8.,    4.],
        [  20.,    3.,   80.,  ..., 2007.,    8.,    4.],
        [  60.,    3.,   68.,  ..., 2008.,    8.,    4.],
        ...,
        [  70.,    3.,   66.,  ..., 2010.,    8.,    4.],
        [  20.,    3.,   68.,  ..., 2010.,    8.,    4.],
        [  20.,    3.,   75.,  ..., 2008.,    8.,    4.]])
tensor([[208500.],
        [181500.],
        [223500.],
        ...,
        [266500.],
        [142125.],
        [147500.]])


(torch.Size([1460, 79]), torch.Size([1460, 1]))

In [17]:
from torch.utils.data import DataLoader, TensorDataset, random_split
from torch.utils.data.sampler import SubsetRandomSampler
dataset = TensorDataset(inputs,targets)



In [18]:
batch_size = 16
test_split = .2
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
train_size = len(train)
indices = list(range(train_size))
split = int(np.floor(test_split * train_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, test_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
test_sampler = SubsetRandomSampler(test_indices)


In [19]:
train_loader = DataLoader(dataset, batch_size=batch_size, 
                                           sampler=train_sampler)
test_loader = DataLoader(dataset, batch_size=batch_size,
                                                sampler=test_sampler)

In [20]:
for xb, yb in train_loader:
    print("inputs:", xb)
    print("targets:", yb)
    break

inputs: tensor([[2.0000e+01, 3.0000e+00, 6.4000e+01,  ..., 2.0100e+03, 8.0000e+00,
         4.0000e+00],
        [9.0000e+01, 3.0000e+00, 4.2000e+01,  ..., 2.0070e+03, 7.0000e+00,
         0.0000e+00],
        [5.0000e+01, 3.0000e+00, 6.7000e+01,  ..., 2.0090e+03, 4.0000e+00,
         4.0000e+00],
        ...,
        [1.6000e+02, 3.0000e+00, 2.4000e+01,  ..., 2.0090e+03, 8.0000e+00,
         4.0000e+00],
        [1.2000e+02, 1.0000e+00, 5.0000e+01,  ..., 2.0080e+03, 1.0000e+00,
         0.0000e+00],
        [2.0000e+01, 3.0000e+00, 9.1000e+01,  ..., 2.0080e+03, 6.0000e+00,
         5.0000e+00]])
targets: tensor([[245350.],
        [150000.],
        [110000.],
        [196500.],
        [ 98000.],
        [174000.],
        [132500.],
        [167000.],
        [137000.],
        [136000.],
        [216500.],
        [181000.],
        [146000.],
        [147000.],
        [328900.],
        [227680.]])


In [21]:
for xb, yb in test_loader:
    print("inputs:", xb)
    print("targets:", yb)
    break

inputs: tensor([[6.0000e+01, 3.0000e+00, 9.4000e+01,  ..., 2.0090e+03, 8.0000e+00,
         4.0000e+00],
        [1.6000e+02, 1.0000e+00, 6.9000e+01,  ..., 2.0100e+03, 8.0000e+00,
         4.0000e+00],
        [1.6000e+02, 4.0000e+00, 2.4000e+01,  ..., 2.0080e+03, 0.0000e+00,
         4.0000e+00],
        ...,
        [1.2000e+02, 1.0000e+00, 3.7000e+01,  ..., 2.0060e+03, 8.0000e+00,
         4.0000e+00],
        [9.0000e+01, 3.0000e+00, 8.0000e+01,  ..., 2.0100e+03, 8.0000e+00,
         4.0000e+00],
        [1.6000e+02, 4.0000e+00, 2.4000e+01,  ..., 2.0060e+03, 8.0000e+00,
         4.0000e+00]])
targets: tensor([[260000.],
        [177500.],
        [151000.],
        [165600.],
        [310000.],
        [135750.],
        [260000.],
        [119000.],
        [556581.],
        [ 91000.],
        [173000.],
        [136500.],
        [181000.],
        [197000.],
        [154300.],
        [137500.]])


In [22]:
input_size = len(input_col)
output_size = len(output_col)

In [23]:
class LRModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = torch.nn.Linear(input_size,output_size)
    
    def forward(self,xb):
        out = self.linear(xb)
        return out
    
    def training_step(self,batch):
        inputs,targets = batch
        out = self(inputs)
        loss = F.l1_loss(input = out,target= targets, size_average=None,reduce=None,reduction='mean')
        return loss
    
    def validation_step(self,batch):
        inputs,targets = batch
        out = self(inputs)
        loss = F.l1_loss(input = out,target= targets, size_average=None,reduce=None,reduction='mean')
        return {'val_loss':loss.detach()}
    
    def validation_epoch_end(self,outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean() 
        return {'val_loss':epoch_loss.item()}
    
    def epoch_end(self,epoch,result,num_epochs):
        if (epoch+1) % 20 == 0 or epoch==num_epochs-1:
            print("Epoch [{}],val_loss:{:.4f}".format(epoch+1,result['val_loss']))
            

In [24]:
model = LRModel()

In [25]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.0024, -0.0518,  0.0668,  0.0678,  0.0171, -0.0583,  0.0587, -0.0545,
          -0.0392, -0.0024, -0.0385, -0.0195, -0.0625,  0.0930,  0.0973, -0.0207,
          -0.0199,  0.1012,  0.0306,  0.0267, -0.0696, -0.0178,  0.0599,  0.0043,
           0.0334,  0.0426, -0.0689, -0.0480, -0.1030, -0.0897,  0.0114,  0.0764,
          -0.0785,  0.1100, -0.0059,  0.0275,  0.0346,  0.0239,  0.0618, -0.0249,
          -0.0638, -0.0387,  0.0418, -0.0361,  0.0658, -0.0879,  0.1092,  0.0388,
          -0.0657,  0.0262, -0.0552,  0.1022,  0.0947, -0.0503,  0.1007,  0.0314,
           0.0624,  0.0806,  0.0894,  0.0213, -0.1010, -0.0786, -0.0122,  0.0121,
           0.0351, -0.1051,  0.0569, -0.0833,  0.1082,  0.1039,  0.0100, -0.0266,
           0.0452, -0.0198, -0.0648,  0.0774, -0.0830, -0.0136,  0.1002]],
        requires_grad=True),
 Parameter containing:
 tensor([0.0076], requires_grad=True)]

In [35]:
def evaluate(model, test_loader):
    outputs = [model.validation_step(batch) for batch in test_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, test_loader, opt_func = torch.optim.ASGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, test_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)
    return history

In [36]:
epochs = 1000
lr = 1e-7
history1 = fit(epochs, lr, model, train_loader, test_loader)

Epoch [20],val_loss:58466.9141
Epoch [40],val_loss:57748.6523
Epoch [60],val_loss:59269.2695
Epoch [80],val_loss:59475.6641
Epoch [100],val_loss:61773.3828
Epoch [120],val_loss:58299.8242
Epoch [140],val_loss:58527.2109
Epoch [160],val_loss:58426.2031
Epoch [180],val_loss:60914.9023
Epoch [200],val_loss:56939.7422
Epoch [220],val_loss:58523.0391
Epoch [240],val_loss:57285.9336
Epoch [260],val_loss:56977.5195
Epoch [280],val_loss:56555.6562
Epoch [300],val_loss:59516.1328
Epoch [320],val_loss:57544.3633
Epoch [340],val_loss:57183.9492
Epoch [360],val_loss:56906.0742
Epoch [380],val_loss:56560.7617
Epoch [400],val_loss:57168.6641
Epoch [420],val_loss:57899.5391
Epoch [440],val_loss:56788.2500
Epoch [460],val_loss:59463.4883
Epoch [480],val_loss:56322.3828
Epoch [500],val_loss:57722.7188
Epoch [520],val_loss:55355.7031
Epoch [540],val_loss:61373.9258
Epoch [560],val_loss:55992.2031
Epoch [580],val_loss:56072.2891
Epoch [600],val_loss:56878.7891
Epoch [620],val_loss:54866.9922
Epoch [640],

In [28]:
def predict_single(inputs, target, model):
    inputs = inputs.unsqueeze(0)
    predictions = model(inputs)                
    prediction = predictions[0].detach()
    print("Input:", input)
    print("Target:", target)
    print("Prediction:", prediction)